In [ ]:
# IMPORTING DATASET 
import pandas as pd
import numpy as np

df = pd.read_csv('C:/Users/User/Downloads/wta_2017_2023.csv')

# prune columns
df =df.drop(['match_num', 'winner_ht', 'winner_ioc', 'loser_ht', 'loser_ioc', 'best_of'], axis=1)

#check for duplicates
df2 = df[df.duplicated()]
df2.head()

In [ ]:
#create new df with only tournament and basic player info for points only

df3 = df[['tourney_id', 'tourney_name', 'tourney_date', 'surface', 'draw_size', 'tourney_level', 'winner_id', 'winner_name', 'loser_id', 'loser_name', 'score', 'round']].copy()

# remove tournaments with no impact on ranking
df3 = df3[df3["tourney_name"].str.contains("BJK Cup") == False]
df3 = df3[df3["tourney_name"].str.contains("Fed Cup") == False]
df3 = df3[df3["tourney_name"].str.contains("United Cup") == False]
df3 = df3[df3["tourney_name"].str.contains("Olympics") == False]

# cleaning tournament names
df3.loc[ df3["tourney_name"] == "Tokyo ", "tourney_name"] = "Tokyo"
df3.loc[ df3["tourney_name"] == "Toronto ", "tourney_name"] = "Toronto"
df3.loc[ df3["tourney_name"] == "'s Hertogenbosch", "tourney_name"] = "s Hertogenbosch"
df3.loc[ df3["tourney_name"] == "s-Hertogenbosch", "tourney_name"] = "s Hertogenbosch"
df3.loc[ df3["tourney_name"] == "Beijing ", "tourney_name"] = "Beijing"

# Preprocessing check
print(df3.isnull().any())

In [ ]:
# Read CSV of tournaments and levels
tournlevel = pd.read_csv('C:/Users/User/Downloads/tournaments - Sheet1 (6).csv')

# Read CSV of levels and points per round
levelpoints = pd.read_csv('C:/Users/User/Downloads/levels, points - Sheet1 (4).csv')
levelpoints.index = ['2000', '1000 (M, 96)', '1000 (M, 64)', '1000', '500 (64)', '500', '250', '125', '1500'] 
levelpoints = levelpoints.fillna(0)

In [ ]:
# Total ranking simulation

global ranking
ranking = {} # overall dictionary

def givepoints(player, tournament, roundd, year):
    if tournament == "Dubai":
        tournament = str(str(year)+"Dubai") # because Doha and Dubai alternate levels
    elif tournament == "Doha":
        tournament = str(str(year)+"Doha")
    tourney_level = tournlevel.loc[tournlevel['Name'] == tournament, 'Level'].iloc[0] #gets tourney level
    points = levelpoints.loc[tourney_level][roundd] # gets points
    if player in ranking.keys():
        ranking[player] += points
    else:
        ranking[player] = points

def simulate(year, num):
    nextyear = year + num - 1
    for ind in df3.index:
        if((str(year) in df3['tourney_id'][ind]) or (str(nextyear) in df3['tourney_id'][ind])):
            if df3['round'][ind] == "F":
                givepoints(df3['winner_name'][ind], df3['tourney_name'][ind], 'Win', year)
            givepoints(df3['loser_name'][ind], df3['tourney_name'][ind], df3['round'][ind], year)

# overall structure of rankings accurate within +- 5 ranks of actual ranking

simulate(2017, 2)

# Creates list from dictionary with only player names
sortedL = sorted(ranking.items(), key = lambda x:x[1], reverse = True)
sT = [item[0] for item in sorted(ranking.items(), key=lambda x: x[1], reverse=True)]
print(sT)

In [ ]:
# Surface-specific ranking simulation

# Defining dictionaries
global r_hard, r_clay, r_grass
r_hard = {}
r_clay = {}
r_grass = {}


# Same function as total give points but with different surfaces
def givepoints(player, tournament, roundd, year, surface):
    if tournament == "Dubai":
        tournament = str(str(year)+"Dubai")
    elif tournament == "Doha":
        tournament = str(str(year)+"Doha")
    tourney_level = tournlevel.loc[tournlevel['Name'] == tournament, 'Level'].iloc[0] #gets tourney level
    points = levelpoints.loc[tourney_level][roundd] # gets points
    if surface == "Hard":
        if player in r_hard.keys():
            r_hard[player] += points
        else:
            r_hard[player] = points
    elif surface == "Clay":
        if player in r_clay.keys():
            r_clay[player] += points
        else:
            r_clay[player] = points
    else:
        if player in r_grass.keys():
            r_grass[player] += points
        else:
            r_grass[player] = points

# Simulates for each match
def simulate(surface, year, num):
    nextyear = year + num - 1
    for ind in df3.index:
        if (((str(year) in df3['tourney_id'][ind]) or (str(nextyear) in df3['tourney_id'][ind])) and (df3['surface'][ind] == surface)):
            if df3['round'][ind] == "F":
                givepoints(df3['winner_name'][ind], df3['tourney_name'][ind], 'Win', year, surface)
            givepoints(df3['loser_name'][ind], df3['tourney_name'][ind], df3['round'][ind], year, surface)

# Simulate with specified surface, starting year, number of years
simulate("Hard", 2017, 2)
simulate("Clay", 2017, 2)
simulate("Grass", 2017, 2)

# Gets lists of only player name in rankings
sH = [item[0] for item in sorted(r_hard.items(), key=lambda x: x[1], reverse=True)]
print(sH)
sC = [item[0] for item in sorted(r_clay.items(), key=lambda x: x[1], reverse=True)]
print("")
sG = [item[0] for item in sorted(r_grass.items(), key=lambda x: x[1], reverse=True)]

rows = zip(sH, sC, sG)

# Code to export rankings to CSV

#from itertools import zip_longest
#import csv
#d = [sH, sC, sG]
#export_data = zip_longest(*d, fillvalue = '')
#with open('surface_rankings.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
#    wr = csv.writer(myfile)
#    wr.writerow(("Hard", "Clay", "Grass"))
#    wr.writerows(export_data)
#myfile.close()

In [ ]:
# Simple model to predict winner of one match
def predict(ranking, tournament, year):
    correct = 0
    count = 0
    for ind in df3.index:
        if ((str(year) in df3['tourney_id'][ind]) and (tournament == df3['tourney_name'][ind])):
            if df3["winner_name"][ind] not in ranking:
                count+=1
                continue
                
            if df3["loser_name"][ind] not in ranking:
                count+=1
                correct+=1
                continue
                
            if(ranking.index(df3["winner_name"][ind])<ranking.index(df3["loser_name"][ind])):
                correct +=1
                count +=1
            else:
                count+=1
                
    return correct/count

# Predicts winners across an entire dataset and returns accuracy
def predictyear(ranking, year, surface):
    correct = 0
    count = 0
    if surface == "All":
        for ind in df3.index:
            if (str(year) in df3['tourney_id'][ind]):
                if df3["winner_name"][ind] not in ranking:
                    count+=1
                    continue

                if df3["loser_name"][ind] not in ranking:
                    count+=1
                    correct+=1
                    continue

                if(ranking.index(df3["winner_name"][ind])<ranking.index(df3["loser_name"][ind])):
                    correct +=1
                    count +=1
                else:
                    count+=1
    else:
        for ind in df3.index:
            if ((str(year) in df3['tourney_id'][ind]) and (surface in df3['surface'][ind])):
                if df3["winner_name"][ind] not in ranking:
                    count+=1
                    continue

                if df3["loser_name"][ind] not in ranking:
                    count+=1
                    correct+=1
                    continue

                if(ranking.index(df3["winner_name"][ind])<ranking.index(df3["loser_name"][ind])):
                    correct +=1
                    count +=1
                else:
                    count+=1
                
    return correct/count
                 

# Lots of tests

print(predictyear(sT, 2019, "All"))
print(predictyear(sT, 2019, "Hard"))
print(predictyear(sT, 2019, "Clay" ))
print(predictyear(sT, 2019, "Grass" )) 
print("")

print(predictyear(sH, 2019, "All"))
print(predictyear(sH, 2019, "Hard"))
print(predictyear(sH, 2019, "Clay" ))
print(predictyear(sH, 2019, "Grass" )) 
print("")

print(predictyear(sC, 2019, "All"))
print(predictyear(sC, 2019, "Hard"))
print(predictyear(sC, 2019, "Clay" ))
print(predictyear(sC, 2019, "Grass" )) 
print("")

print(predictyear(sG, 2019, "All"))
print(predictyear(sG, 2019, "Hard"))
print(predictyear(sG, 2019, "Clay" ))
print(predictyear(sG, 2019, "Grass" )) 
print("") 

In [ ]:
# next: create datasets

# convert dates to string
df3['tourney_date'] = df3['tourney_date'].astype(str)

months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

# define start and end of dataset, add all matches in between
def create_dataset(month, year, length):
    id_month = months[month-1]
    start_matchid = str(year)+id_month
    start_matchid = int(start_matchid)
    start_matchid *= 100
    
    end_matchid = str(year+length)+id_month
    end_matchid = int(end_matchid)
    end_matchid *= 100
    print(start_matchid)
    print(end_matchid)
    
    index=[]
    possible = createset(month, year, length)
    
    dataset = pd.DataFrame()
    for i in possible:
        dataset = pd.concat([dataset, df3[df3['tourney_date'].str.startswith(str(i))]].copy(), ignore_index=True)
    return dataset

# returns possible month year combinations to create dataset
def createset(month, year, length):
    possible = []
    mon = month
    yr = year
    for i in range(12*length):
        if mon > 12:
            mon = 1
            yr += 1
        idd = str(yr)+months[mon-1]
        possible.append(idd)
        mon +=1
    return possible

# DS1
d_j2021_2022 = create_dataset(5, 2021, 1)
d_j2022_2023 = create_dataset(6, 2022, 1)

# DS2 and DS3
d_jan2018_2019 = create_dataset(1, 2018, 1)
d_jan2017_2019 = create_dataset(1, 2017, 2)
d_jan2019_2020 = create_dataset(1, 2019, 1)


In [ ]:
# Create Hard / Clay / Grass datasets out of total datasets
dH2122 = d_j2021_2022[d_j2021_2022.surface == 'Hard'].copy()
dC2122 = d_j2021_2022[d_j2021_2022.surface == 'Clay'].copy()
dG2122 = d_j2021_2022[d_j2021_2022.surface == 'Grass'].copy()

dH2223 = d_j2022_2023[d_j2022_2023.surface == 'Hard'].copy()
dC2223 = d_j2022_2023[d_j2022_2023.surface == 'Clay'].copy()
dG2223 = d_j2022_2023[d_j2022_2023.surface == 'Grass'].copy()

dH1819 = d_jan2018_2019[d_jan2018_2019.surface == 'Hard'].copy()
dC1819 = d_jan2018_2019[d_jan2018_2019.surface == 'Clay'].copy()
dG1819 = d_jan2018_2019[d_jan2018_2019.surface == 'Grass'].copy()

dH1719 = d_jan2017_2019[d_jan2017_2019.surface == 'Hard'].copy()
dC1719 = d_jan2017_2019[d_jan2017_2019.surface == 'Clay'].copy()
dG1719 = d_jan2017_2019[d_jan2017_2019.surface == 'Grass'].copy()

dH19 = d_jan2019_2020[d_jan2019_2020.surface == 'Hard'].copy()
dC19 = d_jan2019_2020[d_jan2019_2020.surface == 'Clay'].copy()
dG19 = d_jan2019_2020[d_jan2019_2020.surface == 'Grass'].copy()

In [ ]:
# Simulate rankings for all datasets

# Same function as before
def givepoints(player, tournament, roundd, year, surface, t):
    
    if tournament == "Dubai":
        tournament = str(str(year)+"Dubai")
    elif tournament == "Doha":
        tournament = str(str(year)+"Doha")
        
    tourney_level = tournlevel.loc[tournlevel['Name'] == tournament, 'Level'].iloc[0] #gets tourney level
    
    points = levelpoints.loc[tourney_level][roundd] # gets points
    
    # Add player to dictionary
    if player not in t.keys():
        t[player] = [0, 0, 0, 0, 0, 0, 0, 0]
    
    # Add number of points and matches depending on surface (index 0 and 1 are hard, 2 and 3 are clay, etc.)
    if surface == "Hard":
        t[player][0] += points
        t[player][1] += 1        
    elif surface == "Clay":
        t[player][2] += points
        t[player][3] += 1
    elif surface == "Grass":
        t[player][4] += points
        t[player][5] += 1
    
    # Total match data
    t[player][6] += points
    t[player][7] += 1
    
    return t

# Simulation function from before
def simulate(dset):
    t_total = {}
    for ind in dset.index:
        surf = dset['surface'][ind]
        year = dset['tourney_id'][ind][:4]
        if dset['round'][ind] == "F":
            t_total = givepoints(dset['winner_name'][ind], dset['tourney_name'][ind], 'Win', year, surf, t_total)
        t_total = givepoints(dset['loser_name'][ind], dset['tourney_name'][ind], dset['round'][ind], year, surf, t_total)
    return t_total

# Organize dictionary into a dataframe and label data
def organize(d):
    o = pd.DataFrame.from_dict(d).transpose()
    o.columns = ['Hard Pts', 'Hard Matches', 'Clay Pts', 'Clay Matches', 'Grass Pts', 'Grass Matches', 'Total Pts', 'Total Matches']
    return o

# Simulate + organize for all total datasets
t_1yr_2019 = simulate(d_jan2018_2019)
t_2yr_2019 = simulate(d_jan2017_2019)
t_test_2019 = simulate(d_jan2019_2020)

t_total = simulate(d_j2021_2022)
t_test = simulate(d_j2022_2023)

df_model= organize(t_total)
df_modelt = organize(t_test)

dfm119 = organize(t_1yr_2019)
dfm219 = organize(t_2yr_2019)
dfmt19 = organize(t_test_2019)

dfmt19.head()
    

In [ ]:
# Add winner and loser surface & total points to dataset
def addpts(d, pts):
    winner_surface_pt = []
    winner_surface_matches = []
    winner_t_pt = []
    winner_t_matches = []
    loser_surface_pt = []
    loser_surface_matches = []
    loser_t_pt = []
    loser_t_matches = []

    for ind in d.index:  
        surf = d['surface'][ind]
        surfpt = surf + " Pts"
        surfmatch = surf + " Matches"

        wplayer = d['winner_name'][ind]
        winner_surface_pt.append(pts.loc[wplayer, surfpt])
        winner_surface_matches.append(pts.loc[wplayer, surfmatch]) # only for specific surface
        winner_t_pt.append(pts.loc[wplayer, "Total Pts"])
        winner_t_matches.append(pts.loc[wplayer, "Total Matches"])
        lplayer = d['loser_name'][ind]
        loser_surface_pt.append(pts.loc[lplayer, surfpt])
        loser_surface_matches.append(pts.loc[lplayer, surfmatch])
        loser_t_pt.append(pts.loc[lplayer, "Total Pts"])
        loser_t_matches.append(pts.loc[lplayer, "Total Matches"])

    d['winner_surface_pt'] = winner_surface_pt
    d['winner_surface_matches'] = winner_surface_matches
    d['winner_t_pt'] = winner_t_pt
    d['winner_t_matches'] = winner_t_matches
    d['loser_surface_pt'] = loser_surface_pt
    d['loser_surface_matches'] = loser_surface_matches
    d['loser_t_pt'] = loser_t_pt
    d['loser_t_matches'] = loser_t_matches
    
    return d

# Add winner and loser hard + clay + grass + total points to datasaet
def addptsALL(d, pts):
    surfaces = ["Hard", "Clay", "Grass"]
    
    winner_Hard_pt = []
    winner_Hard_matches = []
    winner_Clay_pt = []
    winner_Clay_matches = []
    winner_Grass_pt = []
    winner_Grass_matches = []
    winner_t_pt = []
    winner_t_matches = []
    loser_Hard_pt = []
    loser_Hard_matches = []
    loser_Clay_pt = []
    loser_Clay_matches = []
    loser_Grass_pt = []
    loser_Grass_matches = []
    loser_t_pt = []
    loser_t_matches = []

    for ind in d.index:
    
        wplayer = d['winner_name'][ind]
        lplayer = d['loser_name'][ind]
        
        winner_Hard_pt.append(pts.loc[wplayer, "Hard Pts"])
        winner_Hard_matches.append(pts.loc[wplayer, "Hard Matches"])
        winner_Clay_pt.append(pts.loc[wplayer, "Clay Pts"])
        winner_Clay_matches.append(pts.loc[wplayer, "Clay Matches"])
        winner_Grass_pt.append(pts.loc[wplayer, "Grass Pts"])
        winner_Grass_matches.append(pts.loc[wplayer, "Grass Matches"])
        winner_t_pt.append(pts.loc[wplayer, "Total Pts"])
        winner_t_matches.append(pts.loc[wplayer, "Total Matches"])

        loser_Hard_pt.append(pts.loc[lplayer, "Hard Pts"])
        loser_Hard_matches.append(pts.loc[lplayer, "Hard Matches"])
        loser_Clay_pt.append(pts.loc[lplayer, "Clay Pts"])
        loser_Clay_matches.append(pts.loc[lplayer, "Clay Matches"])
        loser_Grass_pt.append(pts.loc[lplayer, "Grass Pts"])
        loser_Grass_matches.append(pts.loc[lplayer, "Grass Matches"])
        loser_t_pt.append(pts.loc[lplayer, "Total Pts"])
        loser_t_matches.append(pts.loc[lplayer, "Total Matches"])
        
    d['winner_hard_pt'] = winner_Hard_pt
    d['winner_hard_matches'] = winner_Hard_matches
    d['winner_clay_pt'] = winner_Clay_pt
    d['winner_clay_matches'] = winner_Clay_matches
    d['winner_grass_pt'] = winner_Grass_pt
    d['winner_grass_matches'] = winner_Grass_matches
    d['winner_t_pt'] = winner_t_pt
    d['winner_t_matches'] = winner_t_matches
    
    d['loser_hard_pt'] = loser_Hard_pt
    d['loser_hard_matches'] = loser_Hard_matches
    d['loser_clay_pt'] = loser_Clay_pt
    d['loser_clay_matches'] = loser_Clay_matches
    d['loser_grass_pt'] = loser_Grass_pt
    d['loser_grass_matches'] = loser_Grass_matches
    d['loser_t_pt'] = loser_t_pt
    d['loser_t_matches'] = loser_t_matches
    
    return d

# Many possible datasets

# surface-specific + total
d_j2122 = addpts(d_j2021_2022.copy(), df_model)
d_j2223 = addpts(d_j2022_2023.copy(), df_modelt)
d_1819 = addpts(d_jan2018_2019.copy(), dfm119)
d_1719 = addpts(d_jan2017_2019.copy(), dfm219)
d_t19 = addpts(d_jan2019_2020.copy(), dfmt19)

# all data
d_2122hcg = addptsALL(d_j2021_2022.copy(), df_model)
d_2223hcg = addptsALL(d_j2022_2023.copy(), df_modelt)
d_1819hcg = addptsALL(d_jan2018_2019.copy(), dfm119)
d_1719hcg = addptsALL(d_jan2017_2019.copy(), dfm219)
d_t19hcg = addptsALL(d_jan2019_2020.copy(), dfmt19)

# hard + total
h2122 = addpts(dH2122.copy(), df_model)
h2223 = addpts(dH2223.copy(), df_modelt)
h1819 = addpts(dH1819.copy(), dfm119)
h1719 = addpts(dH1719.copy(), dfm219)
h19 = addpts(dH19.copy(), dfmt19)

# clay + total
c2122 = addpts(dC2122.copy(), df_model)
c2223 = addpts(dC2223.copy(), df_modelt)
c1819 = addpts(dC1819.copy(), dfm119)
c1719 = addpts(dC1719.copy(), dfm219)
c19 = addpts(dC19.copy(), dfmt19)

# grass + total
g2122 = addpts(dG2122.copy(), df_model)
g2223 = addpts(dG2223.copy(), df_modelt)
g1819 = addpts(dG1819.copy(), dfm119)
g1719 = addpts(dG1719.copy(), dfm219)
g19 = addpts(dG19.copy(), dfmt19)

In [ ]:
# more data cleaning and preprocessing; dropping columns that will not be used in decision tree
def dropcol(d):
    return d.drop(['tourney_id', 'tourney_name', 'tourney_date', 'draw_size', 'tourney_level', 'winner_id', 'loser_id', 'score'], axis=1)

# clean for all datasets
d_j2122 = dropcol(d_j2122)
d_j2223 = dropcol(d_j2223)
d_1819 = dropcol(d_1819)
d_1719 = dropcol(d_1719)
d_t19 = dropcol(d_t19)

d_2122hcg = dropcol(d_2122hcg)
d_2223hcg = dropcol(d_2223hcg)
d_1819hcg = dropcol(d_1819hcg)
d_1719hcg = dropcol(d_1719hcg)
d_t19hcg = dropcol(d_t19hcg)

h2122 = dropcol(h2122)
c2122 = dropcol(c2122)
g2122 = dropcol(g2122)

h2223 = dropcol(h2223)
c2223 = dropcol(c2223)
g2223 = dropcol(g2223)

h1819 = dropcol(h1819)
c1819 = dropcol(c1819)
g1819 = dropcol(g1819)

h1719 = dropcol(h1719)
c1719 = dropcol(c1719)
g1719 = dropcol(g1719)

h19 = dropcol(h19)
c19 = dropcol(c19)
g19 = dropcol(g19)

In [ ]:
# Randomize loser and winner data

from random import randint

def randomize_players(d, datyp, pts):
    # p1 = player 1; p2 = player 2; sp = surface points; sm = surface matches; tp = total points; tm = total matches
    # hp = hard points; hm = hard matches; cp = clay points; cm = clay matches; gp = grass points; gm = grass matches
    p1sp = []
    p1sm = []
    p1tp = []
    p1tm = []
    
    p2sp = []
    p2sm = []
    p2tp = []
    p2tm = []
    
    p1hp = []
    p1hm = []
    p1cp = []
    p1cm = []
    p1gp = []
    p1gm = []
    
    p2hp = []
    p2hm = []
    p2cp = []
    p2cm = []
    p2gp = []
    p2gm = []
    
    winner = [] # winner index column
    
    if datyp == "O": # surface-specific and total
    
        for ind in d.index:
            n = randint(0,1)
            if n == 0:
                p1sp.append(d['winner_surface_pt'][ind])
                p1sm.append(d['winner_surface_matches'][ind])
                p1tp.append(d['winner_t_pt'][ind])
                p1tm.append(d['winner_t_matches'][ind])

                p2sp.append(d['loser_surface_pt'][ind])
                p2sm.append(d['loser_surface_matches'][ind])
                p2tp.append(d['loser_t_pt'][ind])
                p2tm.append(d['loser_t_matches'][ind])

                winner.append(0)

            else:
                p1sp.append(d['loser_surface_pt'][ind])
                p1sm.append(d['loser_surface_matches'][ind])
                p1tp.append(d['loser_t_pt'][ind])
                p1tm.append(d['loser_t_matches'][ind])

                p2sp.append(d['winner_surface_pt'][ind])
                p2sm.append(d['winner_surface_matches'][ind])
                p2tp.append(d['winner_t_pt'][ind])
                p2tm.append(d['winner_t_matches'][ind])

                winner.append(1)

        return pd.DataFrame(list(zip(d['surface'], d['round'], p1sp, p1sm, p1tp, p1tm, p2sp, p2sm, p2tp, p2tm, winner)),
                            columns=['surface', 'round', 'p1_surfpt', 'p1_surfmatch', 'p1_totalpt', 'p1_totalmatch', 
                                    'p2_surfpt', 'p2_surfmatch', 'p2_totalpt', 'p2_totalmatch', 'winner'])
    
    elif datyp == "All": # total only
        
        for ind in d.index:
            n = randint(0,1)
            if n == 0:
                p1tp.append(d['winner_t_pt'][ind])
                p1tm.append(d['winner_t_matches'][ind])

                p2tp.append(d['loser_t_pt'][ind])
                p2tm.append(d['loser_t_matches'][ind])

                winner.append(0)

            else:
                p1tp.append(d['loser_t_pt'][ind])
                p1tm.append(d['loser_t_matches'][ind])

                p2tp.append(d['winner_t_pt'][ind])
                p2tm.append(d['winner_t_matches'][ind])

                winner.append(1)

        return pd.DataFrame(list(zip(d['surface'], d['round'], p1tp, p1tm, p2tp, p2tm, winner)),
                            columns=['surface', 'round', 'p1_totalpt', 'p1_totalmatch',
                                     'p2_totalpt', 'p2_totalmatch', 'winner'])
    
    elif datyp == "Hard":

        surfpt = "Hard Pts"
        surfmatch = "Hard Matches"
        
        for ind in d.index:
            wplayer = d['winner_name'][ind]
            lplayer = d['loser_name'][ind]
            
            n = randint(0,1)
            
            if n == 0:
                p1sp.append(pts.loc[wplayer, surfpt])
                p1sm.append(pts.loc[wplayer, surfmatch])

                p2sp.append(pts.loc[lplayer, surfpt])
                p2sm.append(pts.loc[lplayer, surfmatch])

                winner.append(0)

            else:
                p1sp.append(pts.loc[lplayer, surfpt])
                p1sm.append(pts.loc[lplayer, surfmatch])

                p2sp.append(pts.loc[wplayer, surfpt])
                p2sm.append(pts.loc[wplayer, surfmatch])

                winner.append(1)

        return pd.DataFrame(list(zip(d['surface'], d['round'], p1sp, p1sm, p2sp, p2sm, winner)),
                            columns=['surface', 'round', 'p1_surfpt', 'p1_surfmatch',
                                    'p2_surfpt', 'p2_surfmatch', 'winner'])
    
    elif datyp == "Clay":
        
        surfpt = "Clay Pts"
        surfmatch = "Clay Matches"
        
        for ind in d.index:
            wplayer = d['winner_name'][ind]
            lplayer = d['loser_name'][ind]
            
            n = randint(0,1)
            
            if n == 0:
                p1sp.append(pts.loc[wplayer, surfpt])
                p1sm.append(pts.loc[wplayer, surfmatch])

                p2sp.append(pts.loc[lplayer, surfpt])
                p2sm.append(pts.loc[lplayer, surfmatch])

                winner.append(0)

            else:
                p1sp.append(pts.loc[lplayer, surfpt])
                p1sm.append(pts.loc[lplayer, surfmatch])

                p2sp.append(pts.loc[wplayer, surfpt])
                p2sm.append(pts.loc[wplayer, surfmatch])

                winner.append(1)

        return pd.DataFrame(list(zip(d['surface'], d['round'], p1sp, p1sm, p2sp, p2sm, winner)),
                            columns=['surface', 'round', 'p1_surfpt', 'p1_surfmatch',
                                    'p2_surfpt', 'p2_surfmatch', 'winner'])
    
    elif datyp == "Grass":

        surfpt = "Grass Pts"
        surfmatch = "Grass Matches"
        
        for ind in d.index:
            wplayer = d['winner_name'][ind]
            lplayer = d['loser_name'][ind]
            
            n = randint(0,1)
            
            if n == 0:
                p1sp.append(pts.loc[wplayer, surfpt])
                p1sm.append(pts.loc[wplayer, surfmatch])

                p2sp.append(pts.loc[lplayer, surfpt])
                p2sm.append(pts.loc[lplayer, surfmatch])

                winner.append(0)

            else:
                p1sp.append(pts.loc[lplayer, surfpt])
                p1sm.append(pts.loc[lplayer, surfmatch])

                p2sp.append(pts.loc[wplayer, surfpt])
                p2sm.append(pts.loc[wplayer, surfmatch])

                winner.append(1)

        return pd.DataFrame(list(zip(d['surface'], d['round'], p1sp, p1sm, p2sp, p2sm, winner)),
                            columns=['surface', 'round', 'p1_surfpt', 'p1_surfmatch',
                                    'p2_surfpt', 'p2_surfmatch', 'winner'])
        
    elif datyp == "Every": # all surfaces + total
        
        for ind in d.index:
            wplayer = d['winner_name'][ind]
            lplayer = d['loser_name'][ind]
            
            n = randint(0,1)
            
            if n == 0:
                p1hp.append(d['winner_hard_pt'][ind])
                p1hm.append(d['winner_hard_matches'][ind])
                p1cp.append(d['winner_clay_pt'][ind])
                p1cm.append(d['winner_clay_matches'][ind])
                p1gp.append(d['winner_grass_pt'][ind])
                p1gm.append(d['winner_grass_matches'][ind])
                p1tp.append(d['winner_t_pt'][ind])
                p1tm.append(d['winner_t_matches'][ind])

                p2hp.append(d['loser_hard_pt'][ind])
                p2hm.append(d['loser_hard_matches'][ind])
                p2cp.append(d['loser_clay_pt'][ind])
                p2cm.append(d['loser_clay_matches'][ind])
                p2gp.append(d['loser_grass_pt'][ind])
                p2gm.append(d['loser_grass_matches'][ind])
                p2tp.append(d['loser_t_pt'][ind])
                p2tm.append(d['loser_t_matches'][ind])

                winner.append(0)
            else:
                p2hp.append(d['winner_hard_pt'][ind])
                p2hm.append(d['winner_hard_matches'][ind])
                p2cp.append(d['winner_clay_pt'][ind])
                p2cm.append(d['winner_clay_matches'][ind])
                p2gp.append(d['winner_grass_pt'][ind])
                p2gm.append(d['winner_grass_matches'][ind])
                p2tp.append(d['winner_t_pt'][ind])
                p2tm.append(d['winner_t_matches'][ind])

                p1hp.append(d['loser_hard_pt'][ind])
                p1hm.append(d['loser_hard_matches'][ind])
                p1cp.append(d['loser_clay_pt'][ind])
                p1cm.append(d['loser_clay_matches'][ind])
                p1gp.append(d['loser_grass_pt'][ind])
                p1gm.append(d['loser_grass_matches'][ind])
                p1tp.append(d['loser_t_pt'][ind])
                p1tm.append(d['loser_t_matches'][ind])

                winner.append(1)
                
        return pd.DataFrame(list(zip(d['surface'], d['round'], p1hp, p1hm, p1cp, p1cm, p1gp, p1gm, p1tp, p1tm,
                                     p2hp, p2hm, p2cp, p2cm, p2gp, p2gm, p2tp, p2tm, winner)),
                            columns=['surface', 'round', 'p1_hardpt', 'p1_hardmatch', 'p1_claypt', 'p1_claymatch', 
                                     'p1_grasspt', 'p1_grassmatch', 'p1_totalpt', 'p1_totalmatch',
                                    'p2_hardpt', 'p2_hardmatch', 'p2_claypt', 'p2_claymatch', 'p2_grasspt', 'p2_grassmatch',
                                     'p2_totalpt', 'p2_totalmatch', 'winner'])
                

# Create lot of datasets

# DS1 Train (total data included)
newd_train = randomize_players(d_j2122, "O", df_model)
newd_tra = randomize_players(d_j2122, "All", df_model)
newd_trh = randomize_players(d_j2122, "Hard", df_model)
newd_trc = randomize_players(d_j2122, "Clay", df_model)
newd_trg = randomize_players(d_j2122, "Grass", df_model)
newd_trhcg = randomize_players(d_2122hcg, "Every", df_model)

# DS1 Train surface only
h2122 = randomize_players(h2122, "O", df_model)
c2122 = randomize_players(c2122, "O", df_model)
g2122 = randomize_players(g2122, "O", df_model)

# DS1 Test
newd_test = randomize_players(d_j2223, "O", df_modelt)
newd_ta = randomize_players(d_j2223, "All", df_modelt)
newd_th = randomize_players(d_j2223, "Hard", df_modelt)
newd_tc = randomize_players(d_j2223, "Clay", df_modelt)
newd_tg = randomize_players(d_j2223, "Grass", df_modelt)
newd_thcg = randomize_players(d_2223hcg, "Every", df_modelt)

# DS1 Test surface only
h2223 = randomize_players(h2223, "O", df_modelt)
c2223 = randomize_players(c2223, "O", df_modelt)
g2223 = randomize_players(g2223, "O", df_modelt)

# DS2 Train
d_train1 = randomize_players(d_1819, "O", dfm119)
dtr1a = randomize_players(d_1819, "All", dfm119)
dtr1h = randomize_players(d_1819, "Hard", dfm119)
dtr1c = randomize_players(d_1819, "Clay", dfm119)
dtr1g = randomize_players(d_1819, "Grass", dfm119)
dtr1hcg = randomize_players(d_1819hcg, "Every", dfm119)

# DS2 Train surface only
h1819 = randomize_players(h1819, "O", dfm119)
c1819 = randomize_players(c1819, "O", dfm119)
g1819 = randomize_players(g1819, "O", dfm119)

# DS3 Train
d_train2 = randomize_players(d_1719, "O", dfm219)
dtr2a = randomize_players(d_1719, "All", dfm219)
dtr2h = randomize_players(d_1719, "Hard", dfm219)
dtr2c = randomize_players(d_1719, "Clay", dfm219)
dtr2g = randomize_players(d_1719, "Grass", dfm219)
dtr2hcg = randomize_players(d_1719hcg, "Every", dfm219)

# DS3 Train surface only
h1719 = randomize_players(h1719, "O", dfm219)
c1719 = randomize_players(c1719, "O", dfm219)
g1719 = randomize_players(g1719, "O", dfm219)

# DS2 and DS3 Test
d_test = randomize_players(d_t19, "O", dfmt19)
dta = randomize_players(d_t19, "All", dfmt19)
dth = randomize_players(d_t19, "Hard", dfmt19)
dtc = randomize_players(d_t19, "Clay", dfmt19)
dtg = randomize_players(d_t19, "Grass", dfmt19)
dthcg = randomize_players(d_t19hcg, "Every", dfmt19)

# DS2 and DS3 Test surface only
h19 = randomize_players(h19, "O", dfmt19)
c19 = randomize_players(c19, "O", dfmt19)
g19 = randomize_players(g19, "O", dfmt19)

In [ ]:
# More preprocessing; categorical -> numerical data

# surface
def process_sur(x):
    if(x == 'Hard'):
        return 0
    elif(x == 'Clay'):
        return 1
    elif(x == 'Grass'):
        return 2

# round
def process_rou(x):
    if(x == 'F'):
        return 0
    elif(x=='SF'):
        return 1
    elif(x=='QF'):
        return 2
    elif(x=='R16'):
        return 3
    elif(x=='R32'):
        return 4
    elif(x=='R64'):
        return 5
    elif(x=='R128'):
        return 6
    else:
        return 7

# preprocess
def processcol(d):  
    d["surface"] = d["surface"].apply(process_sur)    
    d["round"] = d["round"].apply(process_rou)  
    return d

# Preprocesses all defined datasets

# DS1 Train
newd_train = processcol(newd_train)
newd_tra = processcol(newd_tra)
newd_trh = processcol(newd_trh)
newd_trc = processcol(newd_trc)
newd_trg = processcol(newd_trg)
newd_trhcg = processcol(newd_trhcg)

# DS1 Train surface only
h2122 = processcol(h2122)
c2122 = processcol(c2122)
g2122 = processcol(g2122)

# DS1 Test
newd_test = processcol(newd_test)
newd_ta = processcol(newd_ta)
newd_th = processcol(newd_th)
newd_tc = processcol(newd_tc)
newd_tg = processcol(newd_tg)
newd_thcg = processcol(newd_thcg)

# DS1 Test surface only
h2223 = processcol(h2223)
c2223 = processcol(c2223)
g2223 = processcol(g2223)

# DS2 Train
d_train1 = processcol(d_train1)
dtr1a = processcol(dtr1a)
dtr1h = processcol(dtr1h)
dtr1c = processcol(dtr1c)
dtr1g = processcol(dtr1g)
dtr1hcg = processcol(dtr1hcg)

# DS2 Train surface only
h1819 = processcol(h1819)
c1819 = processcol(c1819)
g1819 = processcol(g1819)

# DS3 Train
d_train2 = processcol(d_train2)
dtr2a = processcol(dtr2a)
dtr2h = processcol(dtr2h)
dtr2c = processcol(dtr2c)
dtr2g = processcol(dtr2g)
dtr2hcg = processcol(dtr2hcg)

# DS3 Train surface only
h1719 = processcol(h1719)
c1719 = processcol(c1719)
g1719 = processcol(g1719)

# DS2 and DS3 Test
d_test = processcol(d_test)
dta = processcol(dta)
dth = processcol(dth)
dtc = processcol(dtc)
dtg = processcol(dtg)
dthcg = processcol(dthcg)

# DS2 and DS3 Test surface only
h19 = processcol(h19)
c19 = processcol(c19)
g19 = processcol(g19)

In [ ]:
# Create winner column as a separate dataframe for Y train and Y test; remove this column from X train and X test
def create_w(d):
    wnew = d[['winner']].copy()
    d = d.drop(['winner'],axis=1)
    return wnew, d

# DS1 Train
winner_train, newd_train = create_w(newd_train)
wtra, dtra = create_w(newd_tra)
wtrh, dtrh = create_w(newd_trh)
wtrc, dtrc = create_w(newd_trc)
wtrg, dtrg = create_w(newd_trg)
winner_trhcg, newd_trhcg = create_w(newd_trhcg)

# DS1 Train surface only
wh2122, h2122 = create_w(h2122)
wc2122, c2122 = create_w(c2122)
wg2122, g2122 = create_w(g2122)

# DS1 Test
winner_test, newd_test = create_w(newd_test)
wta19, dta19 = create_w(newd_ta)
wth19, dth19 = create_w(newd_th)
wtc19, dtc19 = create_w(newd_tc)
wtg19, dtg19 = create_w(newd_tg)
winner_thcg, newd_thcg = create_w(newd_thcg)

# DS1 Test surface only
wh2223, h2223 = create_w(h2223)
wc2223, c2223 = create_w(c2223)
wg2223, g2223 = create_w(g2223)

# DS2 Train
wtr1, dtr1 = create_w(d_train1)
wtr1a, dtr1a = create_w(dtr1a)
wtr1h, dtr1h = create_w(dtr1h)
wtr1c, dtr1c = create_w(dtr1c)
wtr1g, dtr1g = create_w(dtr1g)
wtr1hcg, dtr1hcg = create_w(dtr1hcg)

# DS2 Train surface only
wh1819, h1819 = create_w(h1819)
wc1819, c1819 = create_w(c1819)
wg1819, g1819 = create_w(g1819)

# DS3 Train
wtr2, dtr2 = create_w(d_train2)
wtr2a, dtr2a = create_w(dtr2a)
wtr2h, dtr2h = create_w(dtr2h)
wtr2c, dtr2c = create_w(dtr2c)
wtr2g, dtr2g = create_w(dtr2g)
wtr2hcg, dtr2hcg = create_w(dtr2hcg)

# DS3 Train surface only
wh1719, h1719 = create_w(h1719)
wc1719, c1719 = create_w(c1719)
wg1719, g1719 = create_w(g1719)

# DS2 and DS3 Test
wt, dt = create_w(d_test)
wta, dta = create_w(dta)
wth, dth = create_w(dth)
wtc, dtc = create_w(dtc)
wtg, dtg = create_w(dtg)
wthcg, dthcg = create_w(dthcg)

# DS2 and DS3 Test surface only 
wh19, h19 = create_w(h19)
wc19, c19 = create_w(c19)
wg19, g19 = create_w(g19)

In [ ]:
# FOR 2021-2022 TRAIN, 2022-2023 TEST; (surface + total)

import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import tree

# x data = all of d_july2023_2022
# y data = winner name only 

# Function to split the dataset
def splitdataset(dataX, dataY):
  
    #Separating the target variable
    X = dataX
    Y = dataY
  
    #Splitting the dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split( 
    X, Y, test_size = 0.2, random_state = 100)
      
    return X, Y, X_train, X_test, y_train, y_test
      
# Function to perform training with giniIndex.
def train_using_gini(X_train, X_test, y_train):
  
    # Creating the classifier object
    clf_gini = DecisionTreeClassifier(criterion = "gini",
            random_state = 100,max_depth=3, min_samples_leaf=5)
  
    # Performing training
    clf_gini.fit(X_train, y_train)
    tree.plot_tree(clf_gini)
    return clf_gini
      
# Function to perform training with entropy.
def tarin_using_entropy(X_train, X_test, y_train):
  
    # Decision tree with entropy
    clf_entropy = DecisionTreeClassifier(
            criterion = "entropy", random_state = 100,
            max_depth = 3, min_samples_leaf = 5)
  
    # Performing training
    clf_entropy.fit(X_train, y_train)
    #tree.plot_tree(clf_entropy)
    return clf_entropy
  
# Function to create predictions
def prediction(X_test, clf_object):
  
    # Predicton on test with giniIndex
    y_pred = clf_object.predict(X_test)
    print("Predicted values:")
    print(y_pred)
    return y_pred
      
# Function to calculate accuracy
def cal_accuracy(y_test, y_pred):
      
    print("Confusion Matrix: ",
        confusion_matrix(y_test, y_pred))
      
    print ("Accuracy : ",
    accuracy_score(y_test,y_pred)*100)
      
    print("Report : ",
    classification_report(y_test, y_pred))
  
   
# Building Phase
X_train = newd_train
X_test = newd_test
y_train = winner_train
y_test = winner_test
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
clf_gini = train_using_gini(X_train, X_test, y_train)
clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
# Operational Phase
print("Results Using Gini Index:")
      
    # Prediction using gini
y_pred_gini = prediction(X_test, clf_gini)
cal_accuracy(y_test, y_pred_gini)
      
print("Results Using Entropy:")
    # Prediction using entropy
y_pred_entropy = prediction(X_test, clf_entropy)
cal_accuracy(y_test, y_pred_entropy)

from sklearn.tree import export_text
tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
print(tree_rules)

In [ ]:
# 2122 all, 2223 all (only total)

# Driver code
def main():
      
    # Building Phase
    X_train = dtra
    X_test = dta19
    y_train = wtra
    y_test = wta19
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# 2122 h, 2223 h

# Driver code
def main():
      
    # Building Phase
    X_train = dtrh
    X_test = dth19
    y_train = wtrh
    y_test = wth19
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# 2122 c, 2223 c

# Driver code
def main():
      
    # Building Phase
    X_train = dtrc
    X_test = dtc19
    y_train = wtrc
    y_test = wtc19
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# 2122 g, 2223 g

# Driver code
def main():
      
    # Building Phase
    X_train = dtrg
    X_test = dtg19
    y_train = wtrg
    y_test = wtg19
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# FOR 2018-2019 TRAIN, 2019-2020 TEST

# Driver code
      
    # Building Phase
X_train = dtr1
X_test = dt
y_train = wtr1
y_test = wt
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
clf_gini = train_using_gini(X_train, X_test, y_train)
clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
print("Results Using Gini Index:")
      
    # Prediction using gini
y_pred_gini = prediction(X_test, clf_gini)
cal_accuracy(y_test, y_pred_gini)
      
print("Results Using Entropy:")
    # Prediction using entropy
y_pred_entropy = prediction(X_test, clf_entropy)
cal_accuracy(y_test, y_pred_entropy)


    
# Calling main function


In [ ]:
from sklearn.tree import export_text
tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
print(tree_rules)

In [ ]:
# 1819 a, 1920 a

# Driver code
def main():
      
    # Building Phase
    X_train = dtr1a
    X_test = dta
    y_train = wtr1a
    y_test = wta
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# 1819 h, 1920 h

# Driver code
def main():
      
    # Building Phase
    X_train = dtr1h
    X_test = dth
    y_train = wtr1h
    y_test = wth
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# 1819 c, 1920 c

# Driver code
def main():
      
    # Building Phase
    X_train = dtr1c
    X_test = dtc
    y_train = wtr1c
    y_test = wtc
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# 1819 g, 1920 g

# Driver code
def main():
      
    # Building Phase
    X_train = dtr1g
    X_test = dtg
    y_train = wtr1g
    y_test = wtg
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# FOR 2017-2019 TRAIN, 2019-2020 TEST

      # Function to perform training with giniIndex.
def train_using_gini(X_train, X_test, y_train):
  
    # Creating the classifier object
    clf_gini = DecisionTreeClassifier(criterion = "gini",
            random_state = 100,max_depth=3, min_samples_leaf=5)
  
    # Performing training
    clf_gini.fit(X_train, y_train)
    tree.plot_tree(clf_gini)
    return clf_gini
      
# Function to perform training with entropy.
def tarin_using_entropy(X_train, X_test, y_train):
  
    # Decision tree with entropy
    clf_entropy = DecisionTreeClassifier(
            criterion = "entropy", random_state = 100,
            max_depth = 3, min_samples_leaf = 5)
  
    # Performing training
    clf_entropy.fit(X_train, y_train)
    #tree.plot_tree(clf_entropy)
    return clf_entropy
  
  
# Function to make predictions
def prediction(X_test, clf_object):
  
    # Predicton on test with giniIndex
    y_pred = clf_object.predict(X_test)
    print("Predicted values:")
    print(y_pred)
    return y_pred
      
# Function to calculate accuracy
def cal_accuracy(y_test, y_pred):
      
    print("Confusion Matrix: ",
        confusion_matrix(y_test, y_pred))
      
    print ("Accuracy : ",
    accuracy_score(y_test,y_pred)*100)
      
    print("Report : ",
    classification_report(y_test, y_pred))
  
# Driver code
def main():
      
    # Building Phase
    X_train = dtr2
    X_test = dt
    y_train = wtr2
    y_test = wt
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()


In [ ]:
# 1719 a, 1920 a

# Driver code
def main():
      
    # Building Phase
    X_train = dtr2a
    X_test = dta
    y_train = wtr2a
    y_test = wta
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# 1719 h, 1920 h

# Driver code
def main():
      
    # Building Phase
    X_train = dtr2h
    X_test = dth
    y_train = wtr2h
    y_test = wth
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# 1719 c, 1920 c

# Driver code
def main():
      
    # Building Phase
    X_train = dtr2c
    X_test = dtc
    y_train = wtr2c
    y_test = wtc
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# 1719 g, 1920 g

# Driver code
def main():
      
    # Building Phase
    X_train = dtr2g
    X_test = dtg
    y_train = wtr2g
    y_test = wtg
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# 2122 tr, 2223 tr, hcg

# Driver code
def main():
      
    # Building Phase
    X_train = newd_trhcg
    X_test = newd_thcg
    y_train = winner_trhcg
    y_test = winner_thcg
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()
    
    '''winner_trhcg, newd_trhcg = create_w(newd_trhcg)
winner_thcg, newd_thcg = create_w(newd_thcg)
wtr1hcg, dtr1hcg = create_w(dtr1hcg)
wtr2hcg, dtr2hcg = create_w(dtr2hcg)
wthcg, dthcg = create_w(dthcg)'''

In [ ]:
# 1819 tr, 1920 tr, hcg

# Driver code
def main():
      
    # Building Phase
    X_train = dtr1hcg
    X_test = dthcg
    y_train = wtr1hcg
    y_test = wthcg
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()
    
    '''
wtr1hcg, dtr1hcg = create_w(dtr1hcg)
wtr2hcg, dtr2hcg = create_w(dtr2hcg)
wthcg, dthcg = create_w(dthcg)'''

In [ ]:
# 1719 tr, 1920 tr, hcg

# Driver code
def main():
      
    # Building Phase
    X_train = dtr2hcg
    X_test = dthcg
    y_train = wtr2hcg
    y_test = wthcg
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()
    
    '''

wtr2hcg, dtr2hcg = create_w(dtr2hcg)
wthcg, dthcg = create_w(dthcg)'''

In [ ]:
# 1719 tr, 1920 tr, hcg

# Driver code
def main():
      
    # Building Phase
    X_train = h2122
    X_test = h2223
    y_train = wh2122
    y_test = wh2223
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
# Calling main function
if __name__=="__main__":
    main()


In [ ]:
# 1719 tr, 1920 tr, hcg

# Driver code
def main():
      
    # Building Phase
    X_train = c2122
    X_test = c2223
    y_train = wc2122
    y_test = wc2223
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# 1719 tr, 1920 tr, hcg

# Driver code
def main():
      
    # Building Phase
    X_train = g2122
    X_test = g2223
    y_train = wg2122
    y_test = wg2223
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)

# Calling main function
if __name__=="__main__":
    main()


In [ ]:
from sklearn.tree import export_text
tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
print(tree_rules)

In [ ]:
# 1719 tr, 1920 tr, hcg

# Driver code
def main():
      
    # Building Phase
    X_train = h1819
    X_test = h19
    y_train = wh1819
    y_test = wh19
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# 1719 tr, 1920 tr, hcg

# Driver code
def main():
      
    # Building Phase
    X_train = c1819
    X_test = c19
    y_train = wc1819
    y_test = wc19
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
    from sklearn.tree import export_text
    tree_rules = export_text(clf_gini, feature_names=list(X_train.columns))
    print(tree_rules)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# 1719 tr, 1920 tr, hcg

# Driver code
def main():
      
    # Building Phase
    X_train = g1819
    X_test = g19
    y_train = wg1819
    y_test = wg19
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# 1719 tr, 1920 tr, hcg

# Driver code
def main():
      
    # Building Phase
    X_train = h1719
    X_test = h19
    y_train = wh1719
    y_test = wh19
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# 1719 tr, 1920 tr, hcg

# Driver code
def main():
      
    # Building Phase
    X_train = c1719
    X_test = c19
    y_train = wc1719
    y_test = wc19
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
# 1719 tr, 1920 tr, hcg

# Driver code
def main():
      
    # Building Phase
    X_train = g1719
    X_test = g19
    y_train = wg1719
    y_test = wg19
    #X, Y, X_train, X_test, y_train, y_test = splitdataset(newd_train, winner_train)
    clf_gini = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # Operational Phase
    print("Results Using Gini Index:")
      
    # Prediction using gini
    y_pred_gini = prediction(X_test, clf_gini)
    cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:")
    # Prediction using entropy
    y_pred_entropy = prediction(X_test, clf_entropy)
    cal_accuracy(y_test, y_pred_entropy)
    
# Calling main function
if __name__=="__main__":
    main()

In [ ]:
import numpy; 
print(numpy.version.version)

In [ ]:
# Import python libraries required in this example:
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np

# Define the network model and its arguments. 
# Set the number of neurons/nodes for each layer:
model = Sequential()
model.add(Dense(16, input_dim=10, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(1, activation='softmax')) 

# Compile the model and calculate its accuracy:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

# Print a summary of the Keras model:
model.summary()

X_train = newd_train
y_train = winner_train
X_test = newd_test
y_test = winner_test

history = model.fit(X_train, y_train, epochs=50, batch_size=10)
y_pred = model.predict(X_test)

#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
    
from sklearn.metrics import accuracy_score
a = accuracy_score(pred,y_test)
print('Accuracy is:', a*100)